In [ ]:
# default_exp series.annotate

# series.annotate
> A GUI created from `ipywidgets` for annotating MRI series types.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from dicomtools.basics import *
from dicomtools.series.preproc import *

from ipywidgets import Button, Layout, HBox, VBox, HTML, Dropdown, Output
from IPython.display import display
import asyncio
from time import sleep

In [ ]:
#export
_labels = [
    't1',
    't2',
    'flair',
    'spgr',
    'swi',
    'dwi',
    'mra',
    'loc',
    'other',
    'unknown'
]

In [ ]:
#export
def _layout_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

_box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='50%')


In [ ]:
#export
def _wait_for_change(widget1, widget2):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget1.on_click(getvalue, remove=True)
        widget2.on_click(getvalue, remove=True)
    widget1.on_click(getvalue)
    widget2.on_click(getvalue)
    return future


In [ ]:
#export
class Annotator():
    "Simple GUI for annotating series from `SeriesDescription` with specified list of `labels`"
    def __init__(self, df, labels=_labels):
        self.data = df
        if 'annotated' not in self.data.columns:
            self.data['annotated'] = 0
        self.labels = labels
        self.accept, self.stop, self.ser_desc, self.label, self.out, self.gui = self._setup()
        
    def _setup(self):
        accept = _layout_button('Accept', 'success')
        stop = _layout_button('Stop', 'danger')
        ser_desc = HTML(description='<strong>Description:</strong>', layout=Layout(width='auto'))
        label = Dropdown(description='<strong>Label:</strong>', options=self.labels, layout=Layout(width='auto'))
        out = Output()
        top_box = HBox(children=[ser_desc, label], layout=_box_layout)
        bottom_box = HBox(children=[accept, stop], layout=_box_layout)
        gui = VBox(children=[top_box, bottom_box, out], layout=_box_layout)
        return accept, stop, ser_desc, label, out, gui
    
    async def _f(self, col='SeriesDescription', label_col='seq_label'):
        if label_col not in self.data.columns:
            self.data[label_col] = np.nan
        grouped = self.data.groupby(col)
        for name, group in grouped:
            if any(group['annotated'] == 1):
                continue
            self.ser_desc.value = name
            self.label.value = group[label_col].unique()[0]
            x = await _wait_for_change(self.accept, self.stop)
            self.out.clear_output()
            if x == 'Accept':
                t = group[label_col].transform(lambda x: self.label.value)
                self.data.loc[t.index, label_col] = t.values
                self.data.loc[t.index, 'annotated'] = 1
            else:
                with self.out:
                    print('Stopping here.')
                break
            with self.out:
                print(f'"{name}" labeled with "{self.label.value}"')
        self.out.clear_output(wait=True)
        with self.out:
            sleep(2)
            print('Done!')
            print(f'{self.data.shape[0] - self.data.annotated.sum()} series remaining.')
        
    def annotate(self):
        asyncio.create_task(self._f())
        display(self.gui)
        
    def save_data(self, fname, ftype='pickle'): 
        if ftype == 'csv':
            self.data.to_csv(fname, index=False)
        elif ftype == 'pickle':
            self.data.to_pickle(fname)
        else:
            raise NameError('File type not supported, please specify "csv" or "pickle".')
            

In [ ]:
show_doc(Annotator)

<h2 id="Annotator" class="doc_header"><code>class</code> <code>Annotator</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Annotator</code>(**`df`**, **`labels`**=*`['t1', 't2', 'flair', 'spgr', 'swi', 'dwi', 'mra', 'loc', 'other', 'unknown']`*)

Simple GUI for annotating series from `SeriesDescription` with specified list of `labels`

In [ ]:
sd = ['ax t1 +c', 'sag t2', 'sag t2', 'ax t1']
lab = ['t1', 'unknown', 'unknown', 't1']
df = pd.DataFrame({'SeriesDescription': sd, 'seq_label': lab})
df

,SeriesDescription,seq_label
0,ax t1 +c,t1
1,sag t2,unknown
2,sag t2,unknown
3,ax t1,t1


In [ ]:
annotator = Annotator(df)
annotator.annotate()

In [ ]:
annotator.data

,SeriesDescription,seq_label,annotated
0,ax t1 +c,t1,0
1,sag t2,unknown,0
2,sag t2,unknown,0
3,ax t1,t1,0
